# Serving PyTorch Models with AI Platform  Custom Prediction Code

AI Platform Online Prediction now supports custom python code in to apply custom prediction routines, including custom (stateful) pre/post processing, and/or models not created by the standard supported frameworks (TensorFlow, Keras, Scikit-learn, XGBoost).

In this notebook, we show how to deploy a model created by [PyTorch](https://pytorch.org/) using AI Platform  Custom Prediction Code

**Note**: You must be whitelisted to use the custom code feature. Please fill out [this google form](https://docs.google.com/forms/d/e/1FAIpQLSc6fxgXQIyA6BDLfCKOJPu5CyCuOB_M_rGTws0629od5mlznw/viewform) to get started.

# Setup

## 1. Preparing your GCP project
* [Create a project on GCP](https://cloud.google.com/resource-manager/docs/creating-managing-projects)
* [Create a Google Cloud Storage Bucket](https://cloud.google.com/storage/docs/quickstart-console)
* [Enable AI Platform Training and Prediction and Compute Engine APIs](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component&_ga=2.217405014.1312742076.1516128282-1417583630.1516128282)

## 2. Preparing your local environment


* [Install Cloud SDK](https://cloud.google.com/sdk/downloads)


Before we start let's install pytorch and gcloud

In [ ]:
!pip install -U google-cloud
!pip install torch

If you are running this notebook in Colab, run the following cell to authenticate your Google Cloud Platform user account

In [ ]:
from google.colab import auth
auth.authenticate_user()

Let's also define the project name, model name, the GCS bucket name that we'll refer to later. 
Replace **<YOUR_PROJECT_ID>**, **<YOUR_BUCKET_NAME>**, and **<YOUR_REGION>** with your GCP project ID, your bucket name, and your region, respectively.

In [ ]:
PROJECT='<YOUR_PROJECT_ID>' 
BUCKET='<YOUR_BUCKET_NAME>'
REGION='<YOUR_REGION>'

!gcloud config set project {PROJECT}
!gcloud config get-value project

## 3. Download iris data
In this example, we want to build a classifier for the simple [iris dataset](https://archive.ics.uci.edu/ml/datasets/iris). So first, we download the data csv file locally.

In [ ]:
!mkdir data
!mkdir models

In [ ]:
import urllib

LOCAL_DATA_DIR = "data/iris.csv"

url_opener = urllib.URLopener()
url_opener.retrieve("https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data", LOCAL_DATA_DIR)

# Part 1: Build a PyTorch NN Classifier

Make sure that pytorch package is [installed](https://pytorch.org/get-started/locally/).

In [ ]:
import torch
from torch.autograd import Variable

print 'PyTorch Version: {}'.format(torch.__version__)

## 1. Load Data 
In this step, we are going to:
1. Load the data to Pandas Dataframe.
2. Convert the class feature (species) from string to a numeric indicator.
3. Split the Dataframe into input feature (xtrain) and target feature (ytrain).

In [ ]:
import pandas as pd

CLASS_VOCAB = ['setosa', 'versicolor', 'virginica']

datatrain = pd.read_csv(LOCAL_DATA_DIR, names=['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'species'])

#change string value to numeric
datatrain.loc[datatrain['species']=='Iris-setosa', 'species']=0
datatrain.loc[datatrain['species']=='Iris-versicolor', 'species']=1
datatrain.loc[datatrain['species']=='Iris-virginica', 'species']=2
datatrain = datatrain.apply(pd.to_numeric)

#change dataframe to array
datatrain_array = datatrain.as_matrix()

#split x and y (feature and target)
xtrain = datatrain_array[:,:4]
ytrain = datatrain_array[:,4]

input_features = xtrain.shape[1]
num_classes = len(CLASS_VOCAB)

print 'Records loaded: {}'.format(len(xtrain))
print 'Number of input features: {}'.format(input_features)
print 'Number of classes: {}'.format(num_classes)

## 2. Set model parameters
You can try different values for **hidden_units** or **learning_rate**.

In [ ]:
hidden_units = 10
learning_rate = 0.1

## 3. Define the PyTorch NN model

Here, we build a a neural network with one hidden layer, and a Softmax output layer for classification.

In [ ]:
model = torch.nn.Sequential(
    torch.nn.Linear(input_features, hidden_units),
    torch.nn.Sigmoid(),
    torch.nn.Linear(hidden_units, num_classes),
    torch.nn.Softmax()
)

loss_metric = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate)

## 4. Train the model
We are going to train the model for **num_epoch** epochs.

In [ ]:
num_epochs = 10000

for epoch in range(num_epochs):
    
    x = Variable(torch.Tensor(xtrain).float())
    y = Variable(torch.Tensor(ytrain).long())

    optimizer.zero_grad()
    
    y_pred = model(x)
    loss = loss_metric(y_pred, y)

    loss.backward()
    optimizer.step()

    if (epoch) % 1000 == 0:
        print 'Epoch [{}/{}] Loss: {}'.format(epoch+1, num_epochs, round(loss.item(),3))
        
print 'Epoch [{}/{}] Loss: {}'.format(epoch+1, num_epochs, round(loss.item(),3))

## 5. Save and load the model

In [ ]:
LOCAL_MODEL_DIR = "models/model.pt"
del model

torch.save(model, LOCAL_MODEL_DIR)
iris_classifier = torch.load(LOCAL_MODEL_DIR)

## 6. Test the loaded model for predictions

In [ ]:
def predict_class(instances):
    instances = torch.Tensor(instances)
    output = iris_classifier(instances)
    _ , predicted = torch.max(output, 1)
    return predicted

Get predictions for the first 5 instances in the dataset

In [ ]:
predicted = predict_class(xtrain[0:5])
print[CLASS_VOCAB[class_index] for class_index in predicted]

Get the classification accuracy on the training data

In [ ]:
import numpy as np
accuracy = round(sum(np.array(predict_class(xtrain)) == ytrain)/float(len(ytrain))*100,2)
print 'Classification accuracy: {}%'.format(accuracy)

## 7. Upload trained model to Cloud Storage

In [ ]:
GCS_MODEL_DIR='models/pytorch/iris_classifier/'

!gsutil -m cp -r {LOCAL_MODEL_DIR} gs://{BUCKET}/{GCS_MODEL_DIR}
!gsutil ls gs://{BUCKET}/{GCS_MODEL_DIR}

# Part 2: Prepare the Custom Prediction Package

1. Implement a model **custom class** for pre/post processing, as well as loading and using your model for prediction.
2. Prepare yout **setup.py** file, to include all the modules and packages you need in your custome model class.

## 1. Create the custom model class
In the **from_path**, you load the pytorch model that you uploaded to GCS. Then in the **predict** method, you use it for prediction.

In [ ]:
%%writefile model.py

import os
import pandas as pd
from google.cloud import storage
import torch

class PyTorchIrisClassifier(object):
    
    def __init__(self, model):
        self._model = model
        self.class_vocab = ['setosa', 'versicolor', 'virginica']
        
    @classmethod
    def from_path(cls, model_dir):
        model_file = os.path.join(model_dir,'model.pt')
        model = torch.load(model_file)    
        return cls(model)

    def predict(self, instances, **kwargs):
        data = pd.DataFrame(instances).as_matrix()
        inputs = torch.Tensor(data)
        outputs = self._model(inputs)
        _ , predicted = torch.max(outputs, 1)
        return [self.class_vocab[class_index] for class_index in predicted]

## 2. Create a setup.py module
Include **pytorch** as a required package, as well as the **model.py** file that includes your custom model class.

In [ ]:
%%writefile setup.py

from setuptools import setup

REQUIRED_PACKAGES = ['torch']

setup(
    name="iris-custom-model",
    version="0.1",
    scripts=["model.py"],
    install_requires=REQUIRED_PACKAGES
)

## 3. Create the package 

This will create a .tar.gz package under /dist directory. The name of the package will be (name)-(version).tar.gz where (name) and (version) are the ones specified in the setup.py.

In [ ]:
!python setup.py sdist

## 4. Uploaded the package to GCS

In [ ]:
GCS_PACKAGE_URI='models/pytorch/packages/iris-custom-model-0.1.tar.gz'

!gsutil cp ./dist/iris-custom-model-0.1.tar.gz gs://{BUCKET}/{GCS_PACKAGE_URI}
!gsutil ls gs://{BUCKET}/{GCS_PACKAGE_URI}

# Part 3: Deploy the Model to AI Platform for Online Predictions

## 1. Create AI Platform model

In [ ]:
MODEL_NAME='torch_iris_classifier'

!gcloud ml-engine models create {MODEL_NAME} --regions {REGION}
!echo ''
!gcloud ml-engine models list | grep 'torch'

## 2. Create AI Platform model version

Once you have your custom package ready, you can specify this as an argument when creating a version resource. Note that you need to provide the path to your package (as package-uris) and also the class name that contains your custom predict method (as model-class).

In [ ]:
MODEL_VERSION='v1'
RUNTIME_VERSION='1.10'
MODEL_CLASS='model.PyTorchIrisClassifier'

!gcloud alpha ml-engine versions create {MODEL_VERSION} --model={MODEL_NAME} \
            --origin=gs://{BUCKET}/{GCS_MODEL_DIR} \
            --runtime-version={RUNTIME_VERSION} \
            --framework='SCIKIT_LEARN' \
            --python-version=2.7 \
            --package-uris=gs://{BUCKET}/{GCS_PACKAGE_URI}\
            --model-class={MODEL_CLASS}

In [ ]:
!gcloud ml-engine versions list --model {MODEL_NAME}

# Part 4: AI Platform Online Prediction

In [ ]:
from googleapiclient import discovery
from oauth2client.client import GoogleCredentials

credentials = GoogleCredentials.get_application_default()
api = discovery.build('ml', 'v1', credentials=credentials,
                      discoveryServiceUrl='https://storage.googleapis.com/cloud-ml/discovery/ml_v1_discovery.json')


def estimate(project, model_name, version, instances):
    
    request_data = {'instances': instances}

    model_url = 'projects/{}/models/{}/versions/{}'.format(project, model_name, version)
    response = api.projects().predict(body=request_data, name=model_url).execute()

    #print response
    
    predictions = response["predictions"]
    return predictions

In [ ]:
instances = [
    [6.8, 2.8, 4.8, 1.4],
    [6. , 3.4, 4.5, 1.6]
]

predictions = estimate(instances=instances
                     ,project=PROJECT
                     ,model_name=MODEL_NAME
                     ,version=MODEL_VERSION)

print(predictions)

# Questions? Feedback?
Feel free to send us an email (cloudml-feedback@google.com) if you run into any issues or have any questions/feedback!